In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

dataset_path = "/content/drive/MyDrive/datasets"

if os.path.exists(dataset_path):
    print("Dataset Loaded Successfully!")
else:
    print("Dataset Not Found! Check your Drive path.")


Dataset Loaded Successfully!


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Set parameters
img_size = (224, 224)
batch_size = 32

# Prepare Data Generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training",
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation",
)

# Load MobileNetV2 (Pre-trained Model)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the main layer of the model.
base_model.trainable = False

# Add a new Fully Connected Layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)  # Binary Classification

# Create a new model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model (first round)
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Fine-Tune (Unfreeze some layers and reduce learning rate)
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(train_generator, validation_data=val_generator, epochs=5)

model.save("glasses_model.h5")

!ls -lh glasses_model.h5


Found 2952 images belonging to 2 classes.
Found 738 images belonging to 2 classes.
Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.8727 - loss: 0.2875 - val_accuracy: 0.9092 - val_loss: 0.2275
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.9517 - loss: 0.1195 - val_accuracy: 0.9038 - val_loss: 0.2263
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.9605 - loss: 0.0928 - val_accuracy: 0.8943 - val_loss: 0.2862
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.9755 - loss: 0.0790 - val_accuracy: 0.9390 - val_loss: 0.1777
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 264s 3s/step - accuracy: 0.9692 - loss: 0.0889 - val_accuracy: 0.9309 - val_loss: 0.2033
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.9722 - loss: 0.0761 - val_accuracy: 0.9539 - val_loss: 0.1278
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.9715 - loss: 0.0846 - val_accuracy: 0.9350 - val_loss: 0.1648
Epoch 8/10
93/93 ━━━━━━━━━━━

-rw-r--r-- 1 root root 29M Feb 26 12:36 glasses_model.h5


In [6]:
from google.colab import files
files.download("glasses_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>